In [7]:
# Import essential libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files

# File upload
uploaded = files.upload()

Saving Transactions.csv to Transactions.csv
Saving Products.csv to Products.csv
Saving Customers.csv to Customers.csv


In [8]:
# Loading the provided datasets
transactions_df = pd.read_csv('/content/Transactions.csv')
customers_df = pd.read_csv('/content/Customers.csv')
products_df = pd.read_csv('/content/Products.csv')

# Displaying a preview of the data
print("Transactions Data:")
print(transactions_df.head(), "\n")

print("Customers Data:")
print(customers_df.head(), "\n")

print("Products Data:")
print(products_df.head())

Transactions Data:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue   Price  
0      300.68  300.68  
1      300.68  300.68  
2      300.68  300.68  
3      601.36  300.68  
4      902.04  300.68   

Customers Data:
  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15 

Products Data:
  ProductID    

In [9]:
# Basic info and missing value analysis
transactions_df.info()
customers_df.info()
products_df.info()

# Check for null values
print("Missing Values in Transactions:")
print(transactions_df.isnull().sum())

print("Missing Values in Customers:")
print(customers_df.isnull().sum())

print("Missing Values in Products:")
print(products_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price            1000 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 54.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
<class 'pandas.core.fr

In [10]:
# Merge datasets for comprehensive analysis
merged_df = transactions_df.merge(customers_df, on='CustomerID').merge(products_df, on='ProductID')

# Preview the merged dataset
print("Merged DataFrame:")
print(merged_df.head())

Merged DataFrame:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving

In [11]:
# Create customer-specific features
customer_features = merged_df.groupby('CustomerID').agg({
    'TotalValue': 'sum',   # Total spend by customer
    'Quantity': 'sum',     # Total quantity purchased
    'ProductID': 'nunique' # Number of unique products bought
}).reset_index()

# Handle missing values
customer_features.fillna(0, inplace=True)

# Preview aggregated data
print("Customer Features:")
print(customer_features.head())

Customer Features:
  CustomerID  TotalValue  Quantity  ProductID
0      C0001     3354.52        12          5
1      C0002     1862.74        10          4
2      C0003     2725.38        14          4
3      C0004     5354.88        23          8
4      C0005     2034.24         7          3


In [12]:
# Set CustomerID as the index for calculations
customer_features.set_index('CustomerID', inplace=True)

# Compute similarity using cosine similarity
similarity_matrix = cosine_similarity(customer_features)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features.index, columns=customer_features.index)

# Preview similarity matrix
print("Customer Similarity Matrix:")
print(similarity_df.head())

Customer Similarity Matrix:
CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000  0.999998  0.999999  1.000000  1.000000  1.000000   
C0002       0.999998  1.000000  1.000000  0.999999  0.999998  0.999996   
C0003       0.999999  1.000000  1.000000  1.000000  0.999999  0.999997   
C0004       1.000000  0.999999  1.000000  1.000000  1.000000  0.999999   
C0005       1.000000  0.999998  0.999999  1.000000  1.000000  1.000000   

CustomerID     C0007     C0008     C0009     C0010  ...     C0191     C0192  \
CustomerID                                          ...                       
C0001       1.000000  0.999999  0.999998  0.999994  ...  1.000000  1.000000   
C0002       0.999997  1.000000  0.999997  0.999999  ...  0.999998  0.999999   
C0003       0.999998  1.000000  0.999997  0.999998  ...  0.999999  0.999999   
C0004       0.999999  1.000000  0.999998  0.999996  ...  1

In [13]:
# Function to fetch top N lookalike customers
def get_top_lookalikes(customer_id, top_n=3):
    if customer_id in similarity_df.index:
        similar_customers = similarity_df[customer_id].sort_values(ascending=False)[1:top_n+1]
        return list(zip(similar_customers.index, similar_customers.values))
    else:
        return []

# Generate lookalikes for the first 20 customers
lookalike_results = {}
for customer_id in customers_df['CustomerID'].head(20):
    lookalike_results[customer_id] = get_top_lookalikes(customer_id)

In [14]:
# Convert results to DataFrame for saving
lookalike_df = pd.DataFrame.from_dict(
    lookalike_results, orient='index', columns=['Lookalike1', 'Lookalike2', 'Lookalike3']
)

# Save to CSV
lookalike_df.to_csv('Lookalike_Model_Output.csv', index_label='CustomerID')

print("Lookalike model results saved as 'Lookalike_Model_Output.csv'")

Lookalike model results saved as 'Lookalike_Model_Output.csv'
